In [1]:
import pandas as pd
import urllib.request
import shutil
import time
import requests
import os
from tqdm import tqdm
from bs4 import BeautifulSoup

In [2]:
lista = [1,2,4,1,8,10,99,1]

In [11]:
print(output.decode())

NameError: name 'output' is not defined

In [12]:
blank = '                                                        '
for i in lista:
    print(i, end='\r')

In [3]:
def get_url_pdfs(contratos):
    links_pdfs = []
    links_outros = []
    urls = contratos.url
    urls = tqdm(urls)
    for i in urls:
        urls.set_description("Processing %s" % i, refresh=True)
        r = requests.get(i)
        soup = BeautifulSoup(r.content, 'lxml')
        pdf = soup.find(id='anexo1').find_all('a')[0]['href']
        if pdf:
            one_pdf = 'http://sistemas.macae.rj.gov.br:84' + pdf
            links_pdfs.append(one_pdf)
        else:
            links_pdfs.append('-')
            links_outros.append(i)
    return links_pdfs, links_outros

In [4]:
def download_file(url, path='./'):
    file_name = path + url.split('/')[-1]
    # Download the file from `url` and save it locally under `file_name`:
    with urllib.request.urlopen(url) as response, open(file_name, 'wb') as out_file:
        shutil.copyfileobj(response, out_file)
    return file_name

In [5]:
def main():
    print('Processo Iniciado ---- ' + time.ctime(time.time()))
    contratos = pd.read_csv('full_table_contratos.csv').head(40)
    links_pdfs, links_outros = get_url_pdfs(contratos)
    contratos['pdf_url_contrato'] = links_pdfs
    contratos['pdf_file_contrato'] = [i.split('/')[-1] for i in links_pdfs]
    contratos.to_csv('full_table_contratos.csv', index=0)

    print('Total de documentos: ' + str(len(links_pdfs)))
    for i, j in enumerate(links_pdfs):
        file = j.split('/')[-1]
        if not os.path.exists(folder + str(file)):
            try:
                download_file(j, path=folder)
                print('----> baixando contrato ' + str(i), end="\r")
                print('----> pdf atual: ' + str(j), end="\r")
            except:
                continue
    print('Captura finalizada ---- ' + time.ctime(time.time()))

In [11]:
contratos = pd.read_csv('full_table_contratos.csv').head(10)

In [12]:
a,b =get_url_pdfs(contratos)


  0%|          | 0/10 [00:00<?, ?it/s]
Processing http://sistemas.macae.rj.gov.br:84/transparencia/default/contratacoes/mostrarcontratos?id=1642:   0%|          | 0/10 [00:00<?, ?it/s]
Processing http://sistemas.macae.rj.gov.br:84/transparencia/default/contratacoes/mostrarcontratos?id=1642:  10%|█         | 1/10 [00:00<00:04,  1.84it/s]
Processing http://sistemas.macae.rj.gov.br:84/transparencia/default/contratacoes/mostrarcontratos?id=1536:  10%|█         | 1/10 [00:00<00:04,  1.84it/s]
Processing http://sistemas.macae.rj.gov.br:84/transparencia/default/contratacoes/mostrarcontratos?id=1536:  20%|██        | 2/10 [00:00<00:03,  2.12it/s]
Processing http://sistemas.macae.rj.gov.br:84/transparencia/default/contratacoes/mostrarcontratos?id=1532:  20%|██        | 2/10 [00:00<00:03,  2.12it/s]
Processing http://sistemas.macae.rj.gov.br:84/transparencia/default/contratacoes/mostrarcontratos?id=1532:  30%|███       | 3/10 [00:01<00:02,  2.36it/s]
Processing http://sistemas.macae.rj.gov.br:8

In [194]:
import re
import operator
import pandas as pd
from docx import Document
from tqdm import tqdm
from itertools import zip_longest #zip two lists that are not of same size


def get_text(filename):
    """
    Transforms docx in machine readable form for parsing.
    :param filename: name of the file, including extension
    :return: document transformed to machine readable form
    """
    file = Document(filename)
    full_text = []
    for i in file.paragraphs:
        full_text.append(i.text)
    doc = '\n'.join(full_text)
    return doc


def get_names(filename, only_first_page=True, chars_first_page=3000):
    """
    Captura expressões em maiúsculo, entre vírgulas, aceitando acentuação (regex1 = r',\s([A-ZÀ-Ú\s]+),') E
        em maiúsculo, entre Sr. e vírgula, aceitando acentuação regex2 = r'\Sr.\s([A-ZÀ-Ú\s]+),' E
        em maiúsculo, entre Sra. e vírgula, aceitando acentuação regex3 = r'\Sra.\s([A-ZÀ-Ú\s]+),'.
    Assume-se que estas regras capturem todos os nomes de pessoas presentes no documento.
    :param doc: documento gerado função que mapeia DOCX (get_text)
    :param only_first_page: flag para indicar se é para pegar apenas a primeira página do documento (proxy: 3000 chars)
    :return: lista com os nomes encontrados e os índices das suas posições
    """
    doc = get_text(filename)
    nomes_bruto = re.findall(r',\s([A-ZÀ-Ú\s]+),'
                             r'|\Sr.\s([A-ZÀ-Ú\s]+),'
                             r'|\Sra.\s([A-ZÀ-Ú\s]+),'
                             , doc)
    nomes, places = [], [] # lista places é para capturar o indice do termo, no texto
    for i in nomes_bruto:
        if i[0]:
            if 6 > len(i[0].split()) > 1:
                nomes.append(i[0])
                places.append(doc.find(i[0]))
        else:
            if 6 > len(i[1].split()) > 1:
                nomes.append(i[1])
                places.append(doc.find(i[1]))
    nomes = [i.rstrip().replace('\n', ' ').replace('\r', ' ') for i in nomes] # the replaces here are for removing line breaks
    names_and_places = dict(zip(nomes, places))

    if only_first_page:
        new = dict()
        for (key, value) in names_and_places.items():
            if value < chars_first_page:
                new[key] = value
        names_and_places = new
    names_and_places = dict(sorted(names_and_places.items(), key=operator.itemgetter(1)))
    return names_and_places


def get_cpf(filename, only_first_page=True, chars_first_page=3000):
    """
    Identifica os CPFs presentes no documento.
    :param doc: documento a ser consumido pela função que mapeia DOCX (get_text)
    :param only_first_page: flag para indicar se é para pegar apenas a primeira página do documento (proxy: 3000 chars)
    :return:
    """
    doc = get_text(filename)
    regex_cpf = r'\d{3}\.\d{3}\.\d{3}\-\d{2}'
    cpfs = re.findall(regex_cpf, doc)
    cpf, places = [], [] # lista places é para capturar o indice do termo, no texto

    for i in cpfs:
        cpf.append(i)
        places.append(doc.find(i))
    cpf_and_places = dict(zip(cpf, places))

    if only_first_page:
        new = dict()
        for (key, value) in cpf_and_places.items():
            if value < chars_first_page:
                new[key] = value
        cpf_and_places = new
    cpf_and_places = dict(sorted(cpf_and_places.items(), key=operator.itemgetter(1)))
    return cpf_and_places


def get_cnpj(filename, only_first_page=True, chars_first_page=3000):
    """
    Identifica os CNPJs presentes no documento.
    :param doc: documento a ser consumido pela função que mapeia DOCX (get_text)
    :param only_first_page: flag para indicar se é para pegar apenas a primeira página do documento (proxy: 3000 chars)
    :return:
    """
    doc = get_text(filename)
    regex_cnpj = r'\d{2}\.\d{3}\.\d{3}\/\d{4}\-\d{2}'
    cnpj = re.findall(regex_cnpj, doc)
    cnp, places = [], [] # lista places é para capturar o indice do termo, no texto

    for i in cnpj:
        cnp.append(i)
        places.append(doc.find(i))
    cnp_and_places = dict(zip(cnp, places))

    if only_first_page:
        new = dict()
        for (key, value) in cnp_and_places.items():
            if value < chars_first_page:
                new[key] = value
        cnp_and_places = new
    cnp_and_places = dict(sorted(cnp_and_places.items(), key=operator.itemgetter(1)))
    return cnp_and_places


def get_names_and_cpfs(filename, only_first_page=True, chars_first_page=3000):
    nms = get_names(filename, only_first_page, chars_first_page)
    cpfs = get_cpf(filename, only_first_page, chars_first_page)

    junto = zip_longest(nms, cpfs, fillvalue='-')
    full = dict(junto)
    return full


def main():
    content = []
    for i in tqdm(numeros_contratos):
        try:
            filename = folder_contratos_docx + str(int(i)) + '.docx'
            item = get_names_and_cpfs(filename, only_first_page=True, chars_first_page=3000)
            content.append(item)
        except:
            content.append('{}')
            pass
    data = {'nro_contrato': list(numeros_contratos), 'content': list(content)}
    nomes_contratos = pd.DataFrame(data)
    nomes_contratos.to_csv("nomes_contratos.csv", index=0)
    return nomes_contratos

In [195]:
contratos = pd.read_csv('full_table_contratos.csv')
numeros_contratos = contratos['nro_contrato']
folder_contratos_docx = '../contratos_word/'

In [198]:
content = []
for i in tqdm(numeros_contratos):
    try:
        filename = folder_contratos_docx + str(int(i)) + '.docx'
        item = get_names_and_cpfs(filename, only_first_page=True, chars_first_page=3000)
        content.append(item)
    except:
        content.append('{}')
        pass

100%|██████████| 1592/1592 [01:27<00:00, 18.26it/s]


In [201]:
data = {'nro_contrato': list(numeros_contratos), 'content': list(content)}
nomes_contratos = pd.DataFrame(data)

In [202]:
df = nomes_contratos
df

,nro_contrato,content
0,1.578659e+09,"{'JÚLIO MARTINS ANTUNES': '119.404.097-72', 'C..."
1,1.568936e+09,"{'JÚLIO MARTINS ANTUNES': '119.404.097-72', 'C..."
2,1.568760e+09,{'MARCELO MARTINS DE PAULA MUSSI': '010.747.51...
3,1.567662e+09,"{'CÉLIO CHAPETA MATTOSO': '176.889.067-68', 'M..."
4,1.565964e+09,"{'JÚLIO MARTINS ANTUNES': '119.404.097-72', 'C..."
...,...,...
1587,1.556591e+09,{'MARCELO MARTINS DE PAULA MUSSI': '010.747.51...
1588,1.442655e+09,{}
1589,1.441422e+09,{}
1590,1.536606e+09,{'GUSTAVO SILVA GUSMÃO DOS SANTOS': '069.386.4...


In [264]:
dff = pd.DataFrame()

for i,j in df.dropna().iterrows():
    try:
        aux1 = pd.DataFrame.from_dict(j[1], orient='index').reset_index()
        aux1['cnt'] = j[0]
        dff = dff.append(aux1, sort=True)
        dff = dff.reset_index(drop=True)
    except AttributeError as error:
        pass

In [265]:
dff.reset_index(drop=True)
dff.rename(columns={'index':'Nome',0:'cpf/cnpj'})

,Nome,cpf/cnpj,cnt
0,JÚLIO MARTINS ANTUNES,119.404.097-72,1.578659e+09
1,CÉLIO CHAPETA MATTOSO,176.889.067-68,1.578659e+09
2,ROGÉRIO SILVA DE OLIVEIRA,-,1.578659e+09
3,JÚLIO MARTINS ANTUNES,119.404.097-72,1.568936e+09
4,CÉLIO CHAPETA MATTOSO,176.889.067-68,1.568936e+09
...,...,...,...
3062,COM FORNECIMENTO DE MATERIAL,-,1.556591e+09
3063,GUSTAVO SILVA GUSMÃO DOS SANTOS,069.386.457-52,1.536606e+09
3064,CARLOS HENRIQUE DA MOTTA SILVA,003.222.547-44,1.536606e+09
3065,BAIRRO VIRGEM SANTA,-,1.536606e+09


,Nome,cpf/cnpj,cnt
0,JÚLIO MARTINS ANTUNES,119.404.097-72,1.578659e+09
1,CÉLIO CHAPETA MATTOSO,176.889.067-68,1.578659e+09
2,ROGÉRIO SILVA DE OLIVEIRA,-,1.578659e+09
3,JÚLIO MARTINS ANTUNES,119.404.097-72,1.568936e+09
4,CÉLIO CHAPETA MATTOSO,176.889.067-68,1.568936e+09
...,...,...,...
3062,COM FORNECIMENTO DE MATERIAL,-,1.556591e+09
3063,GUSTAVO SILVA GUSMÃO DOS SANTOS,069.386.457-52,1.536606e+09
3064,CARLOS HENRIQUE DA MOTTA SILVA,003.222.547-44,1.536606e+09
3065,BAIRRO VIRGEM SANTA,-,1.536606e+09
